### Final Pair Generator

In [2]:
import os
import json
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.transform import Affine

def process_geojson(geojson_path, raster_path, suffix):
    with open(geojson_path) as f:
        data = json.load(f)
    
    file_basename = geojson_path.split('/')[-1].split('.')[0]

    for suffix_f in ['_optical', '_sar']:
        if file_basename.endswith(suffix_f):
            file_basename = file_basename[:-len(suffix_f)]

    with rasterio.open(raster_path) as src:
        xres, yres = src.res
        crop_width_pixels = int(crop_distance / xres)
        crop_height_pixels = int(crop_distance / yres)
        half_w = crop_width_pixels // 2
        half_h = crop_height_pixels // 2

        height, width = src.height, src.width

        for feature in data['features']:
            coords = feature['geometry']['coordinates']
            index = feature['properties']['index']
            row, col = src.index(*coords)  # rasterio returns row, col

            # Desired full crop window offsets
            window_col_off = col - half_w
            window_row_off = row - half_h

            # Calculate the read window (within image bounds)
            read_col_off = max(window_col_off, 0)
            read_row_off = max(window_row_off, 0)
            read_col_end = min(col + half_w, width)
            read_row_end = min(row + half_h, height)

            read_width = read_col_end - read_col_off
            read_height = read_row_end - read_row_off

            # Padding offsets
            pad_left = read_col_off - window_col_off
            pad_top = read_row_off - window_row_off

            try:
                window = Window(read_col_off, read_row_off, read_width, read_height)
                cropped = src.read(window=window)

                # Prepare zero-padded array
                padded = np.zeros((src.count, crop_height_pixels, crop_width_pixels), dtype=src.dtypes[0])
                padded[:, pad_top:pad_top + read_height, pad_left:pad_left + read_width] = cropped

                # Correct transform
                transform = src.transform * Affine.translation(window_col_off, window_row_off)

                out_path = os.path.join(output_dir, f"{file_basename}_index_{index}_{suffix}.tif")

                with rasterio.open(
                    out_path, 'w',
                    driver='GTiff',
                    height=crop_height_pixels,
                    width=crop_width_pixels,
                    count=src.count,
                    dtype=padded.dtype,
                    crs=src.crs,
                    transform=transform
                ) as dst:
                    dst.write(padded)

                print(f"Saved: {out_path}")
                # break
            except Exception as e:
                print(f"Error processing index {index} ({suffix}): {e}")

# Config
path = '/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train'
sar_geojson_path = os.path.join(path, 'keypoint-crps/03_tiepoints_train_01_sar.geojson')
optical_geojson_path = os.path.join(path, 'keypoint-crps/03_tiepoints_train_01_optical.geojson')
sar_path = os.path.join(path, '03_sar_train_01.tif')
optical_path = os.path.join(path, '03_optical_train_01.tif')
output_dir = os.path.join(path, 'keypoint-crps/train-patch-v2/crops-3')
crop_distance = 400  # in meters

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process SAR and Optical
process_geojson(sar_geojson_path, sar_path, 'sar')
process_geojson(optical_geojson_path, optical_path, 'optical')


Saved: /home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-patch-v2/crops-3/03_tiepoints_train_01_index_0_sar.tif
Saved: /home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-patch-v2/crops-3/03_tiepoints_train_01_index_1_sar.tif
Saved: /home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-patch-v2/crops-3/03_tiepoints_train_01_index_2_sar.tif
Saved: /home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-patch-v2/crops-3/03_tiepoints_train_01_index_3_sar.tif
Saved: /home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-patch-v2/crops-3/03_tiepoints_train_01_index_4_sar.tif
Saved: /home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-patch

In [1]:
# import os
# import rasterio
# from rasterio.warp import reproject, Resampling
# from rasterio.enums import Resampling as ResamplingEnums
# from rasterio.transform import from_bounds

# # --- Configuration ---
# optical_path = '/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-ptach-v2/crop-1/02_tiepoints_train_01_index_2_sar.tif'
# sar_path = '/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps/train-ptach-v2/crop-1/02_tiepoints_train_01_index_2_sar.tif'
# output_path = '/home/savvas/SUPER-NAS/USERS/Chirag/PROJECTS/202504-spacenet9/data/spacenet9-void/data/train/keypoint-crps//train-ptach-v2//02_tiepoints_train_01_index_2_resampled.tif'

# # --- Open Optical to get bounds and size ---
# with rasterio.open(optical_path) as optical:
#     optical_bounds = optical.bounds
#     optical_width = optical.width
#     optical_height = optical.height
#     optical_crs = optical.crs

# # --- Open SAR image (source data) ---
# with rasterio.open(sar_path) as sar:
#     sar_data = sar.read()
#     sar_count = sar.count
#     sar_dtype = sar.dtypes[0]

# # --- Create new transform using Optical bounds and SAR dimensions ---
# # This forcibly stretches or shrinks the SAR image into the optical bounds
# new_transform = from_bounds(
#     *optical_bounds,
#     width=sar_data.shape[2],   # width from SAR
#     height=sar_data.shape[1]   # height from SAR
# )

# # --- Save SAR with new georeferencing ---
# new_meta = sar.meta.copy()
# new_meta.update({
#     "transform": new_transform,
#     "crs": optical_crs,
#     "driver": "GTiff"
# })

# with rasterio.open(output_path, 'w', **new_meta) as dst:
#     dst.write(sar_data)

# print(f"✅ SAR saved with optical bounds as: {output_path}")
